In [11]:
import psycopg2
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from plotly.offline import plot 
import plotly.graph_objects as go
import warnings
import datetime
warnings.filterwarnings('ignore')

# import q_credentials.db_secmaster_cloud_cred as db_secmaster_cred
import q_credentials.db_secmaster_cred as db_secmaster_cred
import q_credentials.db_indicator_cred as db_indicator_cred


In [12]:
# connect to our securities_master database
conn_secmaster = psycopg2.connect(host=db_secmaster_cred.dbHost , database=db_secmaster_cred.dbName, user=db_secmaster_cred.dbUser, password=db_secmaster_cred.dbPWD)

In [13]:
conn_indicator = psycopg2.connect(host=db_indicator_cred.dbHost , database=db_indicator_cred.dbName, user=db_indicator_cred.dbUser, password=db_indicator_cred.dbPWD)

In [18]:
sql="select * from indicator"
ind_list=list(pd.read_sql(sql,con=conn_indicator)['name'])
ind_list

['candle_1', 'candle_2', 'candle_3', 'anomaly', 'level']

In [80]:
symbol_id='USD_CHF'#'AAPL'#'BOM500114'
start_date=datetime.datetime(2018,1,1).strftime("%Y-%m-%d")
indicator_name = 'candle_1'

In [81]:
df_all_ind=pd.DataFrame()
for ind in ind_list:
    print(ind)
    sql="select d.date_price as date, d.value from w_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s'" %(symbol_id, ind, start_date)
    df_indicator=pd.read_sql(sql,con=conn_indicator)
    df_indicator.set_index('date',inplace=True)
    df_indicator=pd.concat([df_indicator.drop(['value'], axis=1), df_indicator['value'].apply(pd.Series)], axis=1)
    df_indicator.columns=[ind+"_"+col for col in df_indicator.columns]
    if df_all_ind.empty:
#         df_all_ind=pd.merge(left=df_price, right=df_indicator,on='date')
        df_all_ind=df_indicator
    else:
        df_all_ind=pd.merge(left=df_all_ind, right=df_indicator,on='date')

candle_1
candle_2
candle_3
anomaly
level


In [82]:
df_all_ind.head()

,candle_1_level,candle_1_pattern_name,candle_2_level,candle_2_pattern_name,candle_3_level,candle_3_pattern_name,anomaly_bear,anomaly_body_size,anomaly_body_size_dev,anomaly_body_size_ema,anomaly_bull,anomaly_close,anomaly_gap,anomaly_gap_dev,anomaly_gap_ema,anomaly_gap_signal,anomaly_high,anomaly_low,anomaly_open,anomaly_vol_anomaly,anomaly_vol_anomaly_2,anomaly_vol_dev,anomaly_vol_ema,anomaly_volume,anomaly_wick_bear,anomaly_wick_bear_dev,anomaly_wick_bear_ema,anomaly_wick_bear_signal,anomaly_wick_bull,anomaly_wick_bull_dev,anomaly_wick_bull_ema,anomaly_wick_bull_signal,level_close,level_high,level_level,level_low,level_nearest_resistance_atr_range,level_nearest_resistance_duration,level_nearest_resistance_range,level_nearest_support_atr_range,level_nearest_support_duration,level_nearest_support_range,level_open,level_resistance,level_support
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-05,1.0,DOJI,0.0,,0.0,,0.0,0.00000,1.000000,0.004212,0.0,0.97489,0.00064,-0.830852,0.003784,0.00000,0.97980,0.96994,0.97489,0.664729,0.0,0.335271,130746.460022,86911.0,0.00495,0.356731,0.003648,0.356731,0.00491,-0.192137,0.006078,0.000000,0.97489,0.97980,-1.0,0.96994,1.446194,28,"[0.99779, 0.99281]",0.002480,35.0,"[0.9735, 0.97635]",0.97489,"[[0.99779, 0.99281, 2017-12-08 00:00:00, True]...","[[0.9735, 0.97635, 2017-12-01 00:00:00, True],..."
2018-01-12,10.0,long,0.0,,0.0,,1.0,0.00779,0.601953,0.004863,0.0,0.96758,0.00048,-0.849199,0.003183,0.00000,0.98460,0.96655,0.97537,0.576699,0.0,0.041955,129538.740018,124104.0,0.00103,-0.675324,0.003172,0.000000,0.00923,0.387782,0.006651,0.387782,0.96758,0.98460,-1.0,0.96655,0.492942,42,"[0.97635, 0.9735]",0.515089,637.0,"[0.94208, 0.97773]",0.97537,"[[0.97635, 0.9735, 2017-12-01 00:00:00, False]...","[[0.94208, 0.97773, 2016-04-15 00:00:00, True]..."
2018-01-19,3.0,HAMMER,6.0,BULLISH (Matching Low),0.0,,1.0,0.00383,0.180754,0.004675,0.0,0.96417,0.00042,-0.843321,0.002681,0.00000,0.96882,0.95356,0.96800,0.135619,1.0,0.249705,137149.150924,171396.0,0.01061,1.344916,0.004525,1.344916,0.00082,-0.853329,0.005591,0.000000,0.96417,0.96882,-1.0,0.95356,0.718328,49,"[0.97635, 0.9735]",0.284860,644.0,"[0.94208, 0.97773]",0.96800,"[[0.97635, 0.9735, 2017-12-01 00:00:00, False]...","[[0.94208, 0.97773, 2016-04-15 00:00:00, True]..."
2018-01-26,10.0,long,0.0,,0.0,,1.0,0.02705,2.093831,0.008743,0.0,0.93381,-0.00331,0.184230,0.002795,-0.18423,0.96402,0.92894,0.96086,1.098095,0.0,0.475557,153355.668938,226285.0,0.00487,0.061588,0.004587,0.061588,0.00316,-0.386263,0.005149,0.000000,0.93381,0.96402,0.0,0.92894,1.371702,651,"[0.97773, 0.94208]",NaN,NaN,NaN,0.96086,"[[0.97773, 0.94208, 2016-04-15 00:00:00, False...",NaN
2018-02-02,5.0,DOJI_SPINOFF,6.0,BULLISH (Matching Low),0.0,,1.0,0.00251,0.670166,0.007610,0.0,0.93168,0.00038,-0.838707,0.002356,0.00000,0.93939,0.92551,0.93419,0.433197,0.0,0.353597,166433.547313,225284.0,0.00617,0.265589,0.004875,0.265589,0.00520,0.008124,0.005158,0.008124,0.93168,0.93939,0.0,0.92551,1.568487,658,"[0.97773, 0.94208]",NaN,NaN,NaN,0.93419,"[[0.97773, 0.94208, 2016-04-15 00:00:00, False...",NaN


In [83]:
df_all_ind.rename(columns={'anomaly_close':'close','anomaly_low':'low','anomaly_high':'high','anomaly_open':'open','anomaly_volume':'volume'},inplace=True)

In [84]:
pd.options.display.max_columns = None

In [87]:
df.iloc[-1]

candle_1_level                                                                        0
candle_1_pattern_name                                                                  
candle_2_level                                                                        1
candle_2_pattern_name                                                 BULLISH ENGULFING
candle_3_level                                                                        0
candle_3_pattern_name                                                                  
anomaly_bear                                                                          0
anomaly_body_size                                                                0.0095
anomaly_body_size_dev                                                          0.488406
anomaly_body_size_ema                                                        0.00638267
anomaly_bull                                                                          1
close                           

In [85]:
[[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_support']]

TypeError: 'float' object is not iterable

In [94]:
def level_plot(df):
    try:
        support_ls = [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_support']]
    except:
        support_ls =[]
    try:
        resistance_ls = [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_resistance']]
    except:
        resistance_ls =[]
    end_dt=df.index[-1]
    res_plot_ls=[]
    sup_plot_ls=[]
    for res in resistance_ls[:5]:
        res_plot_ls.append(dict(x0=res[2],x1=end_dt,y0=res[0],y1=res[1],yref='y1',opacity=.2,fillcolor='Red',line=dict(color="black",width=1)))
    for sup in support_ls[:5]:
        sup_plot_ls.append(dict(x0=sup[2],x1=end_dt,y0=sup[0],y1=sup[1],yref='y1',opacity=.2,fillcolor='green',line=dict(color="black",width=1)))
    return (res_plot_ls+sup_plot_ls)

In [95]:
# df.columns

In [96]:
# candle1_plot_ls=[]
# for i,row in df.tail(10).iterrows():
#     if(row['candle_1_pattern_name']):
# #         print(row['candle_1_pattern_name'])
#         print(pd.to_datetime(i), (i+datetime.timedelta(days=1)))
#         candle1_plot_ls.append(dict(x0=i,x1=(i+datetime.timedelta(days=2)),y0=row['high'],y1=row['low'],yref='y1',opacity=.2,fillcolor='blue',line=dict(color="black",width=1)))

In [97]:
# df=df_all_ind.set_index('date')
df=df_all_ind
interested_feature='anomaly_vol_anomaly'

In [98]:
df_candle_1=df[-10:]
df_candle_1=df_candle_1[df_candle_1['candle_1_pattern_name']!='']
df_candle_1['pattern']='1'

df_candle_2=df[-10:]
df_candle_2=df_candle_2[df_candle_2['candle_2_pattern_name']!='']
df_candle_2['pattern']='2'

df_candle_3=df[-10:]
df_candle_3=df_candle_3[df_candle_3['candle_3_pattern_name']!='']
df_candle_3['pattern']='3'

In [99]:
# Plotting
data = [ dict(
    type = 'candlestick',
    open = df.open,
    high = df.high,
    low = df.low,
    close = df.close,
    x = df.index,
    yaxis = 'y1',
    name = 'price'
)]


data.append( dict( x=df.index, y=df.volume,                         
                         marker=dict( color='blue' ),
                         type='bar', yaxis='y2', name='Volume'))

data.append( dict( x=df.index, y=df[interested_feature],                         
                         marker=dict( color='red' ),
                         type='scatter', yaxis='y3', name=interested_feature))

data.append( dict( x=df_candle_1.index, y=df_candle_1['high'],
                         text=df_candle_1['pattern'],
                         mode="markers+text",textfont_size=15,textposition="top center",
                         marker=dict( color='blue' ),
                         type='scatter', yaxis='y1', name='candle_patter_1'))

data.append( dict( x=df_candle_2.index, y=df_candle_2['high'],
                         text=df_candle_2['pattern'],
                         mode="markers+text",textfont_size=15,textposition="top center",
                         marker=dict( color='red' ),
                         type='scatter', yaxis='y1', name='candle_patter_2'))


data.append( dict( x=df_candle_3.index, y=df_candle_3['high'],
                         text=df_candle_3['pattern'],
                         mode="markers+text",textfont_size=15,textposition="top center",
                         marker=dict( color='red' ),
                         type='scatter', yaxis='y1', name='candle_patter_3'))


layout=dict()    
layout['xaxis'] = dict( rangeslider = dict( visible = False ),autorange=True,fixedrange=False,visible=False,type='category')#type='category',
layout['yaxis'] = dict( domain = [0.2, 1],autorange = True,fixedrange=False)
layout['yaxis2'] = dict( domain = [0.0, 0.1],autorange = True,fixedrange=False)
layout['yaxis3'] = dict( domain = [0.1, 0.2],autorange = True,fixedrange=False)
# layout['shapes'] = [dict(x0=start_dt, x1=end_dt, y0=3057.9, y1=3060.85, yref='y1',opacity=.2,line=dict(color="Red",width=1))]#fillcolor='green',line_width=2
layout['shapes'] = level_plot(df)
# layout['shapes'] = candle1_plot_ls

# df.to_csv("stock_price_indicator.csv",index=True)

fig = dict( data=data, layout=layout )

In [100]:
figure=go.FigureWidget(data=data, layout=layout)

In [101]:
def zoom(layout, x_range):
    print(x_range)
    in_view = df.iloc[int(figure.layout.xaxis.range[0]):int(figure.layout.xaxis.range[1])]
    figure.layout.yaxis2.range = [in_view.volume.min(), in_view.volume.max()]
    figure.layout.yaxis3.range = [in_view[interested_feature].min(), in_view[interested_feature].max()]

In [102]:
figure.layout.on_change(zoom, 'xaxis.range')

In [103]:
figure.update_layout(margin=dict(l=20, r=10),paper_bgcolor="LightSteelBlue",width=2200, height=1000)

FigureWidget({
    'data': [{'close': array([0.97489, 0.96758, 0.96417, 0.93381, 0.93168, 0.93932, 0.92734, 0.…